<a href="https://colab.research.google.com/github/PeterS111/GPT-2_DEMO_FINE_TUNING/blob/main/GPT_2_DEMO_FINE_TUNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Dec 11 09:45:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/PeterS111/GPT-2_DEMO_FINE_TUNING  

Cloning into 'GPT-2_DEMO_FINE_TUNING'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [ ]:
cd /content/GPT2_HEAD

/content/GPT2_HEAD


In [ ]:
pip install "transformers==2.7.0"

In [ ]:
## TRAIN WITHOUT VALIDATION
!python run_lm_finetuning.py --output_dir=output  --model_type=gpt2 --model_name_or_path=gpt2 --do_train  --train_data_file=input_data/Shelley_4.txt --overwrite_output_dir --block_size=200 --per_gpu_train_batch_size=1 --save_steps 100 --save_total_limit 1000 --num_train_epochs=5 --logging_steps=50

In [ ]:
## TRAIN WITH VALIDATION
!python run_lm_finetuning.py --output_dir=output  --model_type=gpt2 --model_name_or_path=gpt2 --do_train  --train_data_file=input_data/Shelley_4.txt --overwrite_output_dir --block_size=200 --per_gpu_train_batch_size=1 --save_steps 100 --save_total_limit 1000 --num_train_epochs=5000 --logging_steps=5

To train from checkpoint:
1. Create "output" folder inside the "input data"
2. Get a copy of cached file of input data, eg.: "gpt2_cached_lm_200_Shelley_4.txt"
3. Rename it to: "checkpoint-300_cached_lm_200_Shelley_4.txt" and put in "input_data/output/"
4. Run TRAIN FROM CHECKPOINT WITHOUT VALIDATION

In [ ]:
## TRAIN FROM CHECKPOINT WITHOUT VALIDATION
!python run_lm_finetuning.py --output_dir=output  --model_type=gpt2 --model_name_or_path=output/checkpoint-300 --do_train --train_data_file=input_data/Shelley_4.txt --overwrite_output_dir --block_size=200 --per_gpu_train_batch_size=1 --save_steps 100 --save_total_limit 1000 --num_train_epochs=5000 --logging_steps=50

In [ ]:
## TRAIN FROM CHECKPOINT WITH VALIDATION
!python run_lm_finetuning.py --output_dir=output  --model_type=gpt2 --model_name_or_path=output/checkpoint-200 --do_train --eval_data_file=input_data/Shelley_4a_val.txt --do_eval --evaluate_during_training --train_data_file=input_data/Shelley_4a_train.txt --overwrite_output_dir --block_size=200 --per_gpu_train_batch_size=1 --save_steps 10000 --save_total_limit 1000 --num_train_epochs=5000 --logging_steps=50

In [ ]:
## TEST GENERATION
!python run_serial_generation.py --model_type gpt2 --temperature 1.0 --top_k 50 --top_p 1.0 --model_name_or_path output/checkpoint-500 --length 500 --prompt "The eternal sky " --seed 37 --num_samples 1

In [ ]:
## ZIPPING TRAINED MODEL FOR EXPORT
!tar -czvf "S4_SHM_300.tar.gz" output/checkpoint-300/*

In [ ]:
## REMOVE FILES
!rm -vR /content/GPT2_HEAD/output/checkpoint-300/  

In [ ]:
## MOUNTING GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## EXPORTING FILE TO DRIVE
!cp "S4_SHM_300.tar.gz" "/content/drive/My Drive/"

In [ ]:
## IMPORTING FILE FROM DRIVE
## 1. CREATE OUTPUT FOLDER IF DOESN'T EXIST:
!mkdir output/ 

In [ ]:
## 2. IMPORT THE MODEL:
!cp "/content/drive/My Drive/S4_SHM_300.tar.gz" /content/GPT2_HEAD/output

In [ ]:
## UNZIPPING IMPORTED MODEL AND REMOVING ZIPPED FILE
!tar xf /content/GPT2_HEAD/output/S4_SHM_300.tar.gz
!rm -v /content/GPT2_HEAD/output/S4_SHM_300.tar.gz

In [ ]:
## GENERATION:
# edit: "run_serial_g.py"
!python run_serial_g.py

In [ ]:
cd GPT2_HEAD/

In [ ]:
 !python run_generation.py --model_type gpt2 --temperature 1.0 --top_k 50 --top_p 1.0 --model_name_or_path "/content/GPT2_HEAD/output/checkpoint-300" --length 700 --prompt "The eternal sky " --seed 56 --num_samples 1 
